# 共同研究者を管理する

GRDMプロジェクトの共同研究者の追加や削除、権限の管理を行う。

## 共同編集者の表示
GRDMに登録されている共同研究者を表示します。<br>

・アクセストークン<br>
・プロジェクトID（暫定）<br>
・表示（ボタン）<br>

共同研究者の編集はGRDMのプロジェクトのメニューにあるメンバーから行ってください。<br>
※【注意】メンバーの権限が「読込み/書込み」「管理者」のいずれかに設定されているメンバーでないとリサーチフローは操作できません。<br>

In [ ]:
# 共同編集者を表示する
import os
import sys
import traceback
from requests.exceptions import RequestException

import panel as pn
import pandas as pd
from IPython.display import display, clear_output
from IPython.core.display import Javascript

sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector
from library.utils.widgets import Button, MessageBox
from library.utils.config import message as msg_config
from library.utils.error import InputWarning, UnauthorizedError, NotFoundURLError
from library.utils.storage_provider import grdm
from library.utils.html.button import create_button
from library.utils.input import get_token, get_project_id

script_file_name = "manage_collaborators"
notebook_name = script_file_name+'.ipynb'


class CollaboratorManager(TaskDirector):

    def __init__(self, working_path:str) -> None:
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900
        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

        self.show_col = ShowCollaborator(self._form_box, self._msg_output)

    @TaskDirector.task_cell("1")
    def generateFormScetion(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        # フォーム定義
        try:
            self.show_col.define_form()
            # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
            self.done_task()
        except InputWarning as e:
            self.log.warning(str(e))
        except RequestException as e:
            self.log.error(str(e))
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))


class ShowCollaborator:

    def __init__(self, form_box, message_box:MessageBox) -> None:
        self._form_box = form_box
        self._msg_output = message_box

        # define widgets
        self.link_button = pn.pane.HTML()

    def define_form(self):
        self.token = get_token()
        self.project_id = get_project_id()
        clear_output()

        self.get_collaborators()

    def generate_button(self, project_id):
        return create_button(
            url=grdm.get_collaborator_url(grdm.SCHEME, grdm.DOMAIN, project_id),
            msg=msg_config.get('manage_collaborators', 'go_grdm'),
            target='_blank',
            button_width='500px'
        )

    def get_collaborators(self):
        """メイン処理"""
        try:
            contents = grdm.get_collaborator_list(grdm.SCHEME, grdm.API_DOMAIN, self.token, self.project_id)
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_error(message)
            raise InputWarning(message)
        except NotFoundURLError:
            message = msg_config.get('form', 'project_id_not_exist')
            self._msg_output.update_error(message)
            raise InputWarning(message)
        except RequestException:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(message)
            raise
        except Exception:
            raise

        self.display_table(contents)
        self.link_button.object = self.generate_button(self.project_id)
        self._form_box.append(self.link_button)


    def display_table(self, contents:dict):
        display_permission = {
                "admin": msg_config.get('manage_collaborators', 'admin'),
                "write": msg_config.get('manage_collaborators', 'write'),
                "read": msg_config.get('manage_collaborators', 'read')
            }

        name_list = []
        permission_list = []
        for full_name, permission in contents.items():
            name_list.append(full_name)
            permission_list.append(display_permission[permission])

        df = pd.DataFrame({
            msg_config.get('manage_collaborators', 'name_title'): name_list,
            msg_config.get('manage_collaborators', 'permission_title'): permission_list
        })
        self._form_box.clear()
        table = pn.pane.DataFrame(df, index=False)
        self._form_box.append('# 共同研究者')
        self._form_box.append(table)


CollaboratorManager(working_path=os.path.abspath('__file__')).generateFormScetion()

## サブフローメニューへ

サブフローメニューへアクセスするボタンを表示します

In [ ]:
# サブフローメニューへアクセスするボタンを表示
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector

script_file_name = "manage_collaborators"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()